In [1]:
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append("..")


from llama_index.llms.openai import OpenAI
from src.local_models.embeddings import get_embed_model
from llama_index.core import ServiceContext

from src.data_loader.load_files import load_data
from src.data_loader.chunking import chunk_docs_standalone
from src.data_loader.load_from_dir import rebuild_index
from src.utils import load_prompt

#from llama_index.core import Document
import os, re


from typing import Dict, List
from dotenv import load_dotenv

In [2]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex, SummaryIndex
from llama_index.core import PromptTemplate
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.schema import IndexNode

from llama_index.agent.openai import OpenAIAgent


from llama_index.core.retrievers import RecursiveRetriever
from llama_index.core.response_synthesizers import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

In [3]:
load_dotenv(override=True)

True

### load llm and embed_model

In [4]:
embed_model = get_embed_model(model_name=os.environ['embed_path'],  model_kwargs={'device': 'cpu'}, encode_kwargs = {'normalize_embeddings': True})

llm = OpenAI()
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)



### load documents and get embeddinsg

In [ ]:
documents = load_data(os.environ['data_path'])


In [ ]:
def get_embeddings_from_docs(docs):
    for i, doc in enumerate(docs):
        #apply chunking to documents
        nodes = [chunk_docs_standalone(doc, chunk_size=512, chunk_overlap=50)][0]
        print(len(nodes))

        #build vector index--first time
        vector_index = VectorStoreIndex(nodes, embed_model=embed_model)
        vector_index.storage_context.persist(persist_dir=f"../db_stores/doc_agent_vector_index/idx_{i}")


        #build keyword indexfirst time
        #kw_index = KeywordTableIndex.from_docunments(docs[kw])
        summary_index = SummaryIndex(nodes, embed_model=embed_model)
        summary_index.storage_context.persist(persist_dir=f"../db_stores/doc_agent_summary_index/idx_{i}")

In [ ]:
get_embeddings_from_docs(documents)

### build query engine (document agents)

In [5]:
#load prompt
prompt_str = load_prompt('../prompt_bank/safety_practice.txt')
new_vector_tmpl = PromptTemplate(prompt_str)

In [6]:
mappings = {
    '第一部分': '公共规则：范围、规范性饮用文件、属于和定义、总则、作业基本条件及要求、保证安全的组织措施、保证安全的技术措施、设备巡视、设备操作',
    '第二部分': '常规作业：单一类型作业、带电作业、邻近带点体作业、二次设备作业、架空线路作业、电力电缆作业、高/低压配电网作业',
    '第三部分': '专项作业：试验作业、电气测量作业、水轮机作业、高处作业、密封空间作业、水域作业、焊接及切割作业、动火作业、起重与运输',
    '第四部分': '工器具：安全工器具、带电作业工具、施工机具、电器工具及一般工具',
}

In [14]:
def build_doc_agent_engine(similarity_top_k=None):
    ##Build Document Agent for each Document
    num_docs = 4#len(documents)

    #build agents dict
    agents = {}
    nodes = []
    for i in range(num_docs):
        
        #load from disk
        vector_index = rebuild_index(persist_dir=f'../db_stores/doc_agent_vector_index/idx_{i}', service_context=service_context)
        fn = list(vector_index.ref_doc_info.values())[0].metadata['file_name'].split('.')[0]




        #load from disk
        summary_index = rebuild_index(persist_dir=f'../db_stores/doc_agent_summary_index/idx_{i}', service_context=service_context)



        #define query engines
        vector_query_engine = vector_index.as_query_engine(llm=llm)
        vector_query_engine.update_prompts({'response_synthesizer:text_qa_template': new_vector_tmpl})
        
        #kw_query_engine = kw_index.as_query_engine()
        list_query_engine = summary_index.as_query_engine(llm=llm)
        list_query_engine.update_prompts({'response_synthesizer:text_qa_template': new_vector_tmpl})

        #define tools
        query_engine_tools = [
            QueryEngineTool(
                query_engine=vector_query_engine,
                metadata=ToolMetadata(
                    name="vector_tool",
                    description=(
                        f"Useful for retrieving specific context from {fn}"
                    )
                )
            ),
            QueryEngineTool(
                #query_engine=kw_query_engine,
                query_engine=list_query_engine,
                metadata=ToolMetadata(
                    name="summary_tool",
                    description=(
                        f"Useful for summarization-wise questions about {fn}"
                    )
                )
            )
        ]
        

        #build agents
        agent = OpenAIAgent.from_tools(
            query_engine_tools,
            llm=llm,
            embed_model=embed_model,
            verbose=True,
            
            
            system_prompt=f"""\
                Make sure to respond in Chinese.

                You must ALWAYS use at least one of the tools provided when answering a question; do NOT rely on prior knowledge.

                Please refer to the summary_tool first if you inquire summarization-wise questions about{mappings[fn]},

                If you need to fetch details about {mappings[fn]}, please refer to the vector_tool first.
                """,
        )
        agents[fn] = agent
    
    
        ##Build Composable Retriever over the agents
        #define top-level nodes
        instru =(
            f"This content contains instructions on safety practice regarding {mappings[fn]}, "
            f"Use this index if you need to look up specific facts about {mappings[fn]}, "
            f"Do not use this index if you want to analyze aspects beyond {mappings[fn]} "

        )

        node = IndexNode(
            text=instru, index_id=fn, obj= agent
            )
        nodes.append(node)
    
    #define top-level retriever
    top_vector_index = VectorStoreIndex(objects=nodes, embed_model=embed_model)
    query_engine = top_vector_index.as_query_engine(similarity_top_k=similarity_top_k, verbose=True)
    return query_engine


In [35]:
t = build_doc_agent_engine(similarity_top_k=1)

In [36]:
response = t.query("作业基本条件及要求有哪些")
#高处作业有哪些安全保障措施
#作业基本条件及要求有哪些


Retrieval entering 第二部分: OpenAIAgent
Retrieving from object OpenAIAgent with query 作业基本条件及要求有哪些
Added user message to memory: 作业基本条件及要求有哪些
=== Calling Function ===
Calling function: vector_tool with args: {"input":"作业基本条件及要求有哪些"}
Got output: 作业基本条件及要求包括：
1. 作业环境应满足安全要求。
2. 在进行电力电缆作业时，应核对电缆标志牌的名称是否与工作票相符。
3. 高处砍剪树木时应使用安全带，并不得将安全带系在待砍剪树枝的断口附近或以上。
4. 在电缆隧道、电缆井内应有充足的照明，并有防火、防水、通风的措施。
5. 融冰装置启动前应关闭电源侧高压室大门，禁止无关人员进入。
6. 待融冰的导线或地线位于杆塔同一侧垂直排列时，应先融上层，后融下层。
7. 装设好导线与地线之间的连接线后，应拆除导线接地线，并拉开地线接地刀闸。
8. 未合接地刀闸前不得徒手碰触架空地线引下线、连接电缆、接地刀闸、电缆头等裸露的电气部位。
9. 操作杆塔上的接地刀闸或装、拆连接线时，应戴绝缘手套，使用绝缘操作杆。
10. 裸露的连接线应盘卷放置在绝缘架上，用绝缘护套包好，悬空放置，不能与塔材接触。
11. 危险化学品从业人员应接受安全教育和岗位技术培训，考核合格后方可上岗作业。

Retrieval entering 第三部分: OpenAIAgent
Retrieving from object OpenAIAgent with query 作业基本条件及要求有哪些
Added user message to memory: 作业基本条件及要求有哪些
=== Calling Function ===
Calling function: vector_tool with args: {"input":"作业基本条件及要求有哪些"}
Got output: 作业基本条件及要求包括：
1. 在使用软梯、挂梯或梯头进行移动作业时，每次只允许一人工作，并且在开始工作前要确保梯头的挂钩口被可靠封闭。
2. 在低温或高温环境下进行高处作业时，需要采取保暖和防

In [38]:
#response.source_nodes[0].text

'作业基本条件及要求包括：\n1. 作业环境应满足安全要求。\n2. 在进行电力电缆作业时，应核对电缆标志牌的名称是否与工作票相符。\n3. 高处砍剪树木时应使用安全带，并不得将安全带系在待砍剪树枝的断口附近或以上。\n4. 在电缆隧道、电缆井内应有充足的照明，并有防火、防水、通风的措施。\n5. 融冰装置启动前应关闭电源侧高压室大门，禁止无关人员进入。\n6. 待融冰的导线或地线位于杆塔同一侧垂直排列时，应先融上层，后融下层。\n7. 装设好导线与地线之间的连接线后，应拆除导线接地线，并拉开地线接地刀闸。\n8. 未合接地刀闸前不得徒手碰触架空地线引下线、连接电缆、接地刀闸、电缆头等裸露的电气部位。\n9. 操作杆塔上的接地刀闸或装、拆连接线时，应戴绝缘手套，使用绝缘操作杆。\n10. 裸露的连接线应盘卷放置在绝缘架上，用绝缘护套包好，悬空放置，不能与塔材接触。\n11. 危险化学品从业人员应接受安全教育和岗位技术培训，考核合格后方可上岗作业。'

In [42]:
#print(response.get_formatted_sources())

> Source (Doc id: 41161c7f-7abd-46cb-81d4-c6092776f228): 作业基本条件及要求包括：
1. 作业环境应满足安全要求。
2. 在进行电力电缆作业时，应核对电缆标志牌的名称是否与工作票相符。
3. 高处砍剪树木时应使用安全带，并不得将安全带系在待砍剪树枝的断...

> Source (Doc id: 1167cac7-3138-4e4f-bd19-e62795ff573d): 根据文档内容，作业基本条件及要求包括：
1. 在使用软梯、挂梯或梯头进行移动作业时，每次只允许一人工作，并且在开始工作前要确保梯头的挂钩口被可靠封闭。
2. 在低温或高温环境下进行高处作业时，需...
